In [41]:
import pandas as pd 
import numpy as np
import botometer

In [42]:
df = pd.read_csv('10k_political_tweets.csv', header=None, index_col=0)
column_labels = ['date', 'user_id', 'user_name', 'tweet_id', 'tweet']
df.columns = column_labels
df.head()

,date,user_id,user_name,tweet_id,tweet
0,,,,,
0,2020-10-12 03:48:35,1220170588267958272,TedMazerDS,1315499580344197121,@GadSaad @JoeBiden Most of the country simply ...
1,2020-10-12 03:48:35,587957065,Stop the Insanity,1315499579400425477,@Blue_Texas2020 @BryanRobinAR Bush won’t endor...
2,2020-10-12 03:48:35,763481578689425409,funny gal,1315499579027206149,RT @JYSexton: It’s kind of amazing to watch pe...
3,2020-10-12 03:48:34,874258472,Jonathan Greenberg,1315499576082661381,@JimCarrey Why the Barrett confirm rush?\r\n \...
4,2020-10-12 03:48:34,1293187865820540928,conservative_Q,1315499575587860480,@DavidJHarrisJr You know it’s extremely hard t...


In [43]:
# Sentiment Analysis

In [44]:
import nltk 
from nltk.corpus import twitter_samples
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk import PunktSentenceTokenizer as punkt 
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize

In [45]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daily\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\daily\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [46]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [47]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

generate the tags for each token in the text, and then lemmatize each word using the tag

In [48]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

### Removing noise

* links
* handles
* special chars

In [49]:
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [50]:
stop_words = stopwords.words('english')

In [51]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

## Word Density

In [52]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [53]:
freq_dist_pos = FreqDist(all_pos_words)

## Data Prep

In [54]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)


In [55]:
import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [56]:
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))


Accuracy is: 0.9966666666666667


Call on tweets within DF

In [57]:
tweet_in = "Crying happy tears of joy rn! My republican, christian dad told me over dinner he was planning to vote for Biden"

tokens_in = remove_noise(word_tokenize(tweet_in))
print(classifier.classify(dict([token, True] for token in tokens_in)))

Negative


Loop through df 

In [20]:
for row in df:
    tweet = df['tweet']
    token = remove_noise(word_tokenize(tweet))
    ans = classifier.classify(dict([token, True] for token in tokens))
    df['Sentiment'] = ans

TypeError: expected string or bytes-like object

In [39]:
sentiment = []
for val in df['tweet']:
    token = remove_noise(word_tokenize(val))
    #ans = classifier.classify(dict([token, True] for token in tokens))
    sentiment.append(classifier.classify(dict([token, True] for token in tokens)))

df['Sentiment'] = sentiment



In [70]:
sentiment = []
for val in df['tweet']:
    tokens_in = remove_noise(word_tokenize(val))
    sentiment.append(classifier.classify(dict([token, True] for token in tokens_in)))



In [71]:
df['Sentiment'] = sentiment

In [72]:
countPos = 0 
countNeg = 0

for val in df['Sentiment']:
    if val == 'Positive':
        countPos += 1
    else:
        countNeg += 1
print('Pos: ', countPos)
print('Neg: ', countNeg)

Pos:  5842
Neg:  4158


In [80]:
pd.set_option('display.max_colwidth', -1) 

In [81]:
df.head(30)

,date,user_id,user_name,tweet_id,tweet,Sentiment
0,,,,,,
0,2020-10-12 03:48:35,1220170588267958272,TedMazerDS,1315499580344197121,@GadSaad @JoeBiden Most of the country simply sees Biden as a “not Trump” vote. This election is basically trump ru… https://t.co/Xf16hWoQ5M,Positive
1,2020-10-12 03:48:35,587957065,Stop the Insanity,1315499579400425477,@Blue_Texas2020 @BryanRobinAR Bush won’t endorse Biden. Trump endorsed Jeb’s son for TX land commissioner and he wo… https://t.co/3tKd114KyZ,Positive
2,2020-10-12 03:48:35,763481578689425409,funny gal,1315499579027206149,"RT @JYSexton: It’s kind of amazing to watch people hound Biden over Court packing, a completely legal thing the system allows, while Trump…",Positive
3,2020-10-12 03:48:34,874258472,Jonathan Greenberg,1315499576082661381,@JimCarrey Why the Barrett confirm rush?\r\n \r\nHear Trump announce his plan to steal vote count with newly packed Supre… https://t.co/wcazsPDSSi,Negative
4,2020-10-12 03:48:34,1293187865820540928,conservative_Q,1315499575587860480,@DavidJHarrisJr You know it’s extremely hard to believe these polls when you see so much hacking for Trump at his g… https://t.co/96mO83rhHg,Negative
5,2020-10-12 03:48:34,871448794499620868,Palpatine’s Crappy Clone,1315499574946136064,RT @MrsRabbitResist: Trump supporters threatening to leave America if Biden wins is probably the best thing I’ve heard all year. If any of…,Negative
6,2020-10-12 03:48:33,2364145870,JB Smith,1315499574082011136,@FernandoAmandi Castro-Chavistas overwhelmingly support Biden. Those who fled Castro-Chavistas overwhelmingly suppo… https://t.co/I9hYO1XLhW,Positive
7,2020-10-12 03:48:33,237803067,"VOTE NOVEMBER 3, 2020",1315499573574500352,RT @Politics_Polls: GEORGIA\r\nBiden 47% (+1)\r\nTrump 46%\r\n.\r\n#GAsen:\r\nOssoff (D) 44% (+1)\r\nPerdue (R-inc) 43%\r\nHazel (L) 4%\r\n.\r\n#GAsen Special:\r\nWarnoc…,Positive
8,2020-10-12 03:48:33,1046555609942896640,ayana_unwala,1315499571792015361,"RT @HelenKennedy: Joe Biden’s most recent endorsements: Cindy McCain, Scientific American, Greta Thunberg, the New England Journal of Medic…",Positive


In [87]:
tweet_sent = df[['tweet', 'Sentiment']].copy()

tweet_sent.to_csv('tweet_sent.csv')